In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import numpy as np
import re
from datetime import datetime, date
import calendar
from calendar import monthrange
from datetime import datetime
import pytz

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.supercars.com/calendar?tab=UPCOMING")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('div', {'class': 'col-span-12 md:col-span-4'})
# <div class="col-span-12 md:col-span-4">
driver.quit()




In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = [] 
year = str(datetime.now().year)

tit_prt = 'Australian Supercars Race '
rnd_cnt = 1
titles = []

month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }



In [5]:
len(schedule_details)

12

In [6]:
# <div class="font-inter font-normal normal text-md antialiased text-white px-1 pt-[1px] leading-6">15 - 25 Feb</div>


date_test = schedule_details[0].find('div', {'class':"font-inter font-normal normal text-md antialiased text-white px-1 pt-[1px] leading-6"}).text.split()
date_test

['15', '-', '25', 'Feb']

In [7]:
dates = []

for i in schedule_details:
    day_ints = []
    scrp_daystr = []
    dates = []
    wknd_dates = []
    
    temp_date = i.find('div', {'class':"font-inter font-normal normal text-md antialiased text-white px-1 pt-[1px] leading-6"}).text.split()    
    temp_month = temp_date[-1]
    
    #   Logic for multi-month events
    if '-' in temp_month:
        temp_start_month = temp_month.split('-')[0]
        temp_end_month = temp_month.split('-')[1]
        converted_start_month = month_mapping[temp_start_month]
        converted_end_month = month_mapping[temp_end_month]
#         print('converted_start_month: ' + str(converted_start_month), 'converted_end_month: ' + str(converted_end_month))

        scrp_day = temp_date[1].split('-')
        start_day, end_day = map(int, scrp_day)  # start_day = 31, end_day = 3
#         print('start_day: ' + str(start_day), 'end_day: ' + str(end_day))
        
        start_date = year + '-' + str(converted_start_month).zfill(2) + '-' + str(start_day).zfill(2)      
#         print('start_date: ' + str(start_date))

        end_date = year + '-' + str(converted_end_month).zfill(2) + '-' + str(end_day).zfill(2)
#         print('end_date: ' + str(end_date))

        _, last_day_of_month = monthrange(int(year), converted_start_month)
        days_in_strt_month = np.arange(start_day, last_day_of_month + 1).tolist()
#         print('last_day_of_month: ' + str(last_day_of_month), 'days_in_strt_month: ' + str(days_in_strt_month) ) 
        
        days_in_end_month = np.arange(1, end_day + 1).tolist()
#         print('days_in_end_month: ' + str(days_in_end_month) ) 
            
        for day in days_in_strt_month:
            date_str = year + '-' + str(converted_start_month).zfill(2) + '-' + str(day).zfill(2)
            dates.append(date_str)
            
        for day in days_in_end_month:
            date_str = year + '-' + str(converted_end_month).zfill(2) + '-' + str(day).zfill(2)
            dates.append(date_str)
            
   


#         logic for single month events

    else:  
        converted_month = month_mapping[temp_month]
#         print('converted_month: ' + str(converted_month))
        
        start_day = int(temp_date[0])
        end_day = int(temp_date[2])
#         start_day, end_day = map(int, scrp_day)  # start_day = 22, end_day = 25
#         print('start_day: ' + str(start_day), 'end_day: ' + str(end_day))
        
#         start_date = year + '-' + str(converted_month).zfill(2) + '-' + str(start_day).zfill(2)      
# #         print('start_date: ' + str(start_date))

#         end_date = year + '-' + str(converted_month).zfill(2) + '-' + str(end_day).zfill(2)
# #         print('end_date: ' + str(end_date))
        
        if len(range(start_day, end_day + 1)) > 6:
            for day in range(start_day, end_day + 1)[5:]:
                date_str = year + '-' + str(converted_month).zfill(2) + '-' + str(day).zfill(2)
                dates.append(date_str)
        
        if len(range(start_day, end_day + 1)) <= 5:
            for day in range(start_day, end_day + 1):
                date_str = year + '-' + str(converted_month).zfill(2) + '-' + str(day).zfill(2)
                dates.append(date_str)

#     print('Dates:', ', '.join(dates))
    
    
    for date in dates:
        date_obj = datetime.strptime(date, '%Y-%m-%d')

        year, month, day = date_obj.year, date_obj.month, date_obj.day
        if calendar.weekday(year, month, day) >= 5:
            wknd_dates.append(date)
#     print('wknd_dates: ' + ', '.join(wknd_dates))
    
    for date in wknd_dates:
        year, month, day = date.split('-')
        day = int(day) # Remove leading zero if present
        month = int(month) # Remove leading zero if present
        days.append(day)
        months.append(month)

# <div class="font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4">Bathurst, NSW</div>
# <div class="font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4">Bathurst, NSW</div>
# <div target="_blank" class="flex flex-row items-center gap-1.5"><div class="font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4">Melbourne, VIC</div></div>
    
    if len(wknd_dates) > 2:
        temp_circuit = i.find('div', {'class':'font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4'}).text
        circuits.extend([temp_circuit] * 2)

        tit = tit_prt + str(rnd_cnt)
        titles.append(tit) 
        rnd_cnt += 1
        tit = tit_prt + str(rnd_cnt)
        titles.append(tit) 
        rnd_cnt += 1
        
        desc = i.find('div', {'class':'font-inter font-normal normal text-base antialiased z-[3] pt-2 md:pt-3 text-dark-grey-2'}).text
        descs.append(desc + ' Race 1')
        descs.append(desc + ' Race 2')
        
    
    
    elif len(wknd_dates) == 2:   
        temp_circuit = i.find('div', {'class':'font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4'}).text
        circuits.extend([temp_circuit] * 2)

        tit = tit_prt + str(rnd_cnt)
        titles.append(tit) 
        rnd_cnt += 1
        tit = tit_prt + str(rnd_cnt)
        titles.append(tit) 
        rnd_cnt += 1
        
        
# <div class="font-inter font-normal normal text-base antialiased z-[3] pt-2 md:pt-3 text-dark-grey-2">2024 Bathurst SuperFest</div>
# <div class="font-inter font-normal normal text-base antialiased z-[3] pt-2 md:pt-3 text-dark-grey-2">2024 Melbourne SuperSprint</div>
        
        desc = i.find('div', {'class':'font-inter font-normal normal text-base antialiased z-[3] pt-2 md:pt-3 text-dark-grey-2'}).text
        descs.append(desc + ' Race 1')
        descs.append(desc + ' Race 2')
        
    elif len(wknd_dates) == 1:
        temp_circuit = i.find('div', {'class':'font-inter font-normal normal text-md antialiased leading-6 text-light-grey-4'}).text
        circuits.append(temp_circuit)

        tit = tit_prt + str(rnd_cnt)
        rnd_cnt += 1
        titles.append(tit) 

        desc = i.find('div', {'class':'font-inter font-normal normal text-base antialiased z-[3] pt-2 md:pt-3 text-dark-grey-2'}).text
        descs.append(desc)


    
    

#### If you need to handle the case where the range spans over a year, you can add logic to update the year accordingly.

# # Checking if the range spans over a year
# if converted_start_month > converted_end_month:
#     next_year = str(int(year) + 1)
#     for month in range(1, converted_end_month + 1):
#         start_date = next_year + '-' + str(month).zfill(2) + '-' + str(start_day).zfill(2)
#         _, last_day_of_month = monthrange(int(next_year), month)
#         days_in_month = np.arange(start_day, last_day_of_month + 1).tolist()
        
#         for day in days_in_month:
#             date_str = next_year + '-' + str(month).zfill(2) + '-' + str(day).zfill(2)
#             dates.append(date_str)
    
#     print('Dates:', ', '.join(dates))

In [8]:
len(titles)

24

In [9]:
len(circuits)

24

In [10]:
len(descs)

24

In [11]:
len(months)

24

In [12]:
len(days)

24

In [13]:
year

'2024'

In [14]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [15]:
Aussie_Supercars = Calendar()
Aussie_Supercars.add('prodid', '-//My calendar product//example.com//')
Aussie_Supercars.add('version', '2.0')
uid = 110

In [16]:
 
for title, circuit, day, month, desc in zip(titles, circuits, days, months, descs):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(int(year), month, day, 13, 0, 0, tzinfo=pytz.timezone('Australia/Sydney')))
    ievent.add('dtend', datetime(int(year), month, day, 15, 0, 0, tzinfo=pytz.timezone('Australia/Sydney')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    Aussie_Supercars.add_component(ievent)

In [17]:
len(Aussie_Supercars)

2

In [18]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'Aussie Supercars.ics'), 'wb')
f.write(Aussie_Supercars.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
